# **Montaje de Google Drive en entorno de Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Instalación de Librerías**

In [ ]:
!pip install streamlit -q  #streamlit es una biblioteca que facilita la creación de aplicaciones web interactivas para el análisis de datos y el ML.
!pip install streamlit-lottie #Permite integrar animaciones Lottie en aplicaciones web construidas con streamlit
!pip install Pillow # Se utiliza para abrir, manipular y guardar imágenes en una variedad de formatos.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 37.6 MB/s eta 0:00:00


## Aplicación

In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image #Importa la clase Image del módulo PIL (Python Imaging Library), que es parte de la biblioteca Pillow.
import numpy as np #Proporciona soporte para matrices multidimensionales.
import joblib #facilita la serialización y deserialización de objetos
import cv2

def segmentacion_img(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def extraccion_caracteristicas(image):
    segmented = segmentacion_img(image)
    shape_features = cv2.HuMoments(cv2.moments(segmented)).flatten()
    color_features = cv2.mean(image)[:3]
    features = np.hstack((shape_features, color_features))
    return features

def cargar_imagen_prediccion(img):
    img = np.array(img)
    img = cv2.resize(img, (150, 150))
    return extraccion_caracteristicas(img)

# Cargar el modelo entrenado
model_svc = joblib.load('/content/drive/MyDrive/clasificacion_micorrizas.pkl')

# Configuración de la interfaz
st.title("Clasificador de Micorrizas")

st.markdown("Proyecto Integrador\n-Jordy Loor\n-Diego Ortiz\n-Abraham Jimenez\n-Leonardo Paredes")

# Subir archivo
uploaded_file = st.file_uploader("Cargar imagen", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Mostrar imagen cargada
    st.image(uploaded_file, caption='Imagen cargada', use_column_width=True)

    # Preprocesar la imagen
    imagen_procesada = cargar_imagen_prediccion(Image.open(uploaded_file))

    # Hacer predicción usando el modelo SVC
    prediction = model_svc.predict([imagen_procesada]) #Realiza una predicción utilizando un modelo de clasificación (model_svc en este caso).
    probabilities = model_svc.predict_proba([imagen_procesada])[0]   #Obtiene las probabilidades de las distintas clases para la entrada proporcionada.

    if prediction[0] == 0:
        st.write("La imagen corresponde a una ectomicorriza.")
        st.write(f"Probabilidad de Ectomicorriza: {probabilities[0]:.2f}")
        st.write(f"Probabilidad de Endomicorriza: {probabilities[1]:.2f}")
    elif prediction[0] == 1:
        st.write("La imagen corresponde a una endomicorriza.")
        st.write(f"Probabilidad de Ectomicorriza: {probabilities[0]:.2f}")
        st.write(f"Probabilidad de Endomicorriza: {probabilities[1]:.2f}")

Writing app.py


# **Ejecución de la app de Streamlit y exposición a través de un túnel público**

In [ ]:
!streamlit run app.py & npx localtunnel --port 8080




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.171.55:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) 